In [0]:
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.rdd import portable_hash
from pyspark.statcounter import StatCounter
from pyspark.sql.types import StringType,IntegerType,FloatType
from pyspark.sql.functions import *
from pyspark.sql.functions import udf
from pyspark.sql import functions as F
from pyspark.sql.functions import to_timestamp, current_timestamp, col,expr,unix_timestamp,round,when,hour
import os
import json
%matplotlib inline

from datetime import datetime
from matplotlib import pyplot as plt

In [0]:
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("FAF").getOrCreate()
sc=spark.sparkContext
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.5 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
#read dataSet 2012
df_2012 = spark.read.csv("/FileStore/tables/FAF4_5_1.csv",header=True, inferSchema=True)
#Filter the dataset to domestic freight goods and select the attributes I need to work on
df_2012_domistic=df_2012.where(col("trade_type")==1)
df_2012_domistic=df_2012_domistic.select("dms_orig","dms_dest","dms_mode","sctg2","wgt_dist","tons_2012","value_2012","tmiles_2012")

#read dataSet 2013
df_2013 = spark.read.csv("/FileStore/tables/FAF4_5_1_2013.csv",header=True, inferSchema=True)
#Filter the dataset to domestic freight goods , dropping the export and import attributes and rename the common attributes names to avoid duplicate columns after join
df_2013_domistic=df_2013.where(col("trade_type")==1)
df_2013_domistic=df_2013_domistic.drop("fr_orig","fr_dest","fr_inmode","fr_outmode","trade_type")
df_2013_domistic=df_2013_domistic.withColumnRenamed('dms_orig', 'dms_orig_13').withColumnRenamed('dms_dest', 'dms_dest_13').withColumnRenamed('dms_mode', 'dms_mode_13').withColumnRenamed('sctg2', 'sctg2_13').withColumnRenamed('wgt_dist', 'wgt_dist_13')

#read dataSet 2014
df_2014 = spark.read.csv("/FileStore/tables/FAF4_5_1_2014.csv",header=True, inferSchema=True)
#Filter the dataset to domestic freight goods , dropping the export and import attributes and rename the common attributes names to avoid duplicate columns after join
df_2014_domistic=df_2014.where(col("trade_type")==1)
df_2014_domistic=df_2014_domistic.drop("fr_orig","fr_dest","fr_inmode","fr_outmode","trade_type")
df_2014_domistic=df_2014_domistic.withColumnRenamed('dms_orig', 'dms_orig_14').withColumnRenamed('dms_dest', 'dms_dest_14').withColumnRenamed('dms_mode', 'dms_mode_14').withColumnRenamed('sctg2', 'sctg2_14').withColumnRenamed('wgt_dist', 'wgt_dist_14')

#read dataSet 2015
df_2015 = spark.read.csv("/FileStore/tables/FAF4_5_1_2015.csv",header=True, inferSchema=True)
#Filter the dataset to domestic freight goods , dropping the export and import attributes and rename the common attributes names to avoid duplicate columns after join
df_2015_domistic=df_2015.where(col("trade_type")==1)
df_2015_domistic=df_2015_domistic.drop("fr_orig","fr_dest","fr_inmode","fr_outmode","trade_type")
df_2015_domistic=df_2015_domistic.withColumnRenamed('dms_orig', 'dms_orig_15').withColumnRenamed('dms_dest', 'dms_dest_15').withColumnRenamed('dms_mode', 'dms_mode_15').withColumnRenamed('sctg2', 'sctg2_15').withColumnRenamed('wgt_dist', 'wgt_dist_15')

#read dataSet 2016
df_2016 = spark.read.csv("/FileStore/tables/FAF4_5_1_2016.csv",header=True, inferSchema=True)
#Filter the dataset to domestic freight goods , dropping the export and import attributes and rename the common attributes names to avoid duplicate columns after join
df_2016_domistic=df_2016.where(col("trade_type")==1)
df_2016_domistic=df_2016_domistic.drop("fr_orig","fr_dest","fr_inmode","fr_outmode","trade_type")
df_2016_domistic=df_2016_domistic.withColumnRenamed('dms_orig', 'dms_orig_16').withColumnRenamed('dms_dest', 'dms_dest_16').withColumnRenamed('dms_mode', 'dms_mode_16').withColumnRenamed('sctg2', 'sctg2_16').withColumnRenamed('wgt_dist', 'wgt_dist_16')

#read dataSet 2017
df_2017 = spark.read.csv("/FileStore/tables/FAF4_5_1_2017.csv",header=True, inferSchema=True)
#Filter the dataset to domestic freight goods , dropping the export and import attributes and rename the common attributes names to avoid duplicate columns after join
df_2017_domistic=df_2017.where(col("trade_type")==1)
df_2017_domistic=df_2017_domistic.drop("fr_orig","fr_dest","fr_inmode","fr_outmode","trade_type")
df_2017_domistic=df_2017_domistic.withColumnRenamed('dms_orig', 'dms_orig_17').withColumnRenamed('dms_dest', 'dms_dest_17').withColumnRenamed('dms_mode', 'dms_mode_17').withColumnRenamed('sctg2', 'sctg2_17').withColumnRenamed('wgt_dist', 'wgt_dist_17')

#read dataSet 2018
df_2018 = spark.read.csv("/FileStore/tables/FAF4_5_1_2018.csv",header=True, inferSchema=True)
#Filter the dataset to domestic freight goods , dropping the export and import attributes and rename the common attributes names to avoid duplicate columns after join
df_2018_domistic=df_2018.where(col("trade_type")==1)
df_2018_domistic=df_2018_domistic.drop("fr_orig","fr_dest","fr_inmode","fr_outmode","trade_type")
df_2018_domistic=df_2018_domistic.withColumnRenamed('dms_orig', 'dms_orig_18').withColumnRenamed('dms_dest', 'dms_dest_18').withColumnRenamed('dms_mode', 'dms_mode_18').withColumnRenamed('sctg2', 'sctg2_18').withColumnRenamed('wgt_dist', 'wgt_dist_18')


In [0]:
#Inner join between six dataSets with conditions of origin , destination , mode of transportation , commodity type and the distance

cond_13=[df_2012_domistic.dms_orig == df_2013_domistic.dms_orig_13,df_2012_domistic.dms_dest == df_2013_domistic.dms_dest_13,df_2012_domistic.dms_mode == df_2013_domistic.dms_mode_13,df_2012_domistic.sctg2 == df_2013_domistic.sctg2_13,df_2012_domistic.wgt_dist == df_2013_domistic.wgt_dist_13]

df_12_13 =df_2012_domistic.join(df_2013_domistic,cond_13 ).drop("dms_orig_13","dms_dest_13","dms_mode_13","sctg2_13","wgt_dist_13")


cond_14=[df_12_13.dms_orig == df_2014_domistic.dms_orig_14,df_12_13.dms_dest == df_2014_domistic.dms_dest_14,df_12_13.dms_mode == df_2014_domistic.dms_mode_14,df_12_13.sctg2 == df_2014_domistic.sctg2_14,df_12_13.wgt_dist == df_2014_domistic.wgt_dist_14]

df_13_14 =df_12_13.join(df_2014_domistic,cond_14 ).drop("dms_orig_14","dms_dest_14","dms_mode_14","sctg2_14","wgt_dist_14")


cond_15=[df_13_14.dms_orig == df_2015_domistic.dms_orig_15,df_13_14.dms_dest == df_2015_domistic.dms_dest_15,df_13_14.dms_mode == df_2015_domistic.dms_mode_15,df_13_14.sctg2 == df_2015_domistic.sctg2_15,df_13_14.wgt_dist == df_2015_domistic.wgt_dist_15]

df_13_14_15 =df_13_14.join(df_2015_domistic,cond_15 ).drop("dms_orig_15","dms_dest_15","dms_mode_15","sctg2_15","wgt_dist_15")


cond_16=[df_13_14_15.dms_orig == df_2016_domistic.dms_orig_16,df_13_14_15.dms_dest == df_2016_domistic.dms_dest_16,df_13_14_15.dms_mode == df_2016_domistic.dms_mode_16,df_13_14_15.sctg2 == df_2016_domistic.sctg2_16,df_13_14_15.wgt_dist == df_2016_domistic.wgt_dist_16]

df_13_14_15_16 =df_13_14_15.join(df_2016_domistic,cond_16 ).drop("dms_orig_16","dms_dest_16","dms_mode_16","sctg2_16","wgt_dist_16")


cond_17=[df_13_14_15_16.dms_orig == df_2017_domistic.dms_orig_17,df_13_14_15_16.dms_dest == df_2017_domistic.dms_dest_17,df_13_14_15_16.dms_mode == df_2017_domistic.dms_mode_17,df_13_14_15_16.sctg2 == df_2017_domistic.sctg2_17,df_13_14_15_16.wgt_dist == df_2017_domistic.wgt_dist_17]

df_13_14_15_16_17 =df_13_14_15_16.join(df_2017_domistic,cond_17 ).drop("dms_orig_17","dms_dest_17","dms_mode_17","sctg2_17","wgt_dist_17")


cond_18=[df_13_14_15_16_17.dms_orig == df_2018_domistic.dms_orig_18,df_13_14_15_16_17.dms_dest == df_2018_domistic.dms_dest_18,df_13_14_15_16_17.dms_mode == df_2018_domistic.dms_mode_18,df_13_14_15_16_17.sctg2 == df_2018_domistic.sctg2_18,df_13_14_15_16_17.wgt_dist == df_2018_domistic.wgt_dist_18]

df_13_14_15_16_17_18 =df_13_14_15_16_17.join(df_2018_domistic,cond_18 ).drop("dms_orig_18","dms_dest_18","dms_mode_18","sctg2_18","wgt_dist_18")

In [0]:
df_13_14_15_16_17_18.printSchema()


root
-- dms_orig: integer (nullable = true)
-- dms_dest: integer (nullable = true)
-- dms_mode: integer (nullable = true)
-- sctg2: integer (nullable = true)
-- wgt_dist: double (nullable = true)
-- tons_2012: double (nullable = true)
-- value_2012: double (nullable = true)
-- tmiles_2012: double (nullable = true)
-- tons_2013: double (nullable = true)
-- value_2013: double (nullable = true)
-- tmiles_2013: double (nullable = true)
-- curval_2013: double (nullable = true)
-- tons_2014: double (nullable = true)
-- value_2014: double (nullable = true)
-- tmiles_2014: double (nullable = true)
-- curval_2014: double (nullable = true)
-- tons_2015: double (nullable = true)
-- value_2015: double (nullable = true)
-- tmiles_2015: double (nullable = true)
-- curval_2015: double (nullable = true)
-- tons_2016: double (nullable = true)
-- value_2016: double (nullable = true)
-- tmiles_2016: double (nullable = true)
-- curval_2016: double (nullable = true)
-- tons_2017: double (nullable = true)
-- value_2017: double (nullable = true)
-- tmiles_2017: double (nullable = true)
-- curval_2017: double (nullable = true)
-- tons_2018: double (nullable = true)
-- value_2018: double (nullable = true)
-- tmiles_2018: double (nullable = true)
-- curval_2018: double (nullable = true)

In [0]:
#calculation of the growth rate for each commodity in each region-to-region from 2013 to 2018
df_with_growth_rate=df_13_14_15_16_17_18.withColumn("tons_GrowthRate_12_13",((col("tons_2013") - col("tons_2012")) / (col("tons_2012"))) * 100 )
df_with_growth_rate=df_with_growth_rate.withColumn("tons_GrowthRate_13_14",((col("tons_2014") - col("tons_2013")) / (col("tons_2013"))) * 100 )
df_with_growth_rate=df_with_growth_rate.withColumn("tons_GrowthRate_14_15",((col("tons_2015") - col("tons_2014")) / (col("tons_2014"))) * 100 )
df_with_growth_rate=df_with_growth_rate.withColumn("tons_GrowthRate_15_16",((col("tons_2016") - col("tons_2015")) / (col("tons_2015"))) * 100 )
df_with_growth_rate=df_with_growth_rate.withColumn("tons_GrowthRate_16_17",((col("tons_2017") - col("tons_2016")) / (col("tons_2016"))) * 100 )
df_with_growth_rate=df_with_growth_rate.withColumn("tons_GrowthRate_17_18",((col("tons_2018") - col("tons_2017")) / (col("tons_2017"))) * 100 )

#Without null valuas
df_with_growth_rate=df_with_growth_rate.where(col("tons_GrowthRate_13_14").isNotNull())

In [0]:
#calculation of the average for the growth rate from 2013 to 2018
df_with_growth_rate=df_with_growth_rate.withColumn("Avg_tons_GrowthRate_12_18",((col("tons_GrowthRate_12_13") + col("tons_GrowthRate_13_14") + col("tons_GrowthRate_14_15") + col("tons_GrowthRate_15_16") + col("tons_GrowthRate_16_17") + col("tons_GrowthRate_17_18")) /6))

In [0]:
df_with_growth_rate.columns

Out[8]: ['dms_orig',
 'dms_dest',
 'dms_mode',
 'sctg2',
 'wgt_dist',
 'tons_2012',
 'value_2012',
 'tmiles_2012',
 'tons_2013',
 'value_2013',
 'tmiles_2013',
 'curval_2013',
 'tons_2014',
 'value_2014',
 'tmiles_2014',
 'curval_2014',
 'tons_2015',
 'value_2015',
 'tmiles_2015',
 'curval_2015',
 'tons_2016',
 'value_2016',
 'tmiles_2016',
 'curval_2016',
 'tons_2017',
 'value_2017',
 'tmiles_2017',
 'curval_2017',
 'tons_2018',
 'value_2018',
 'tmiles_2018',
 'curval_2018',
 'tons_GrowthRate_12_13',
 'tons_GrowthRate_13_14',
 'tons_GrowthRate_14_15',
 'tons_GrowthRate_15_16',
 'tons_GrowthRate_16_17',
 'tons_GrowthRate_17_18',
 'Avg_tons_GrowthRate_12_18']

In [0]:
#This is the growth rate for goods tons from 2013 to each year until 2018 for each commodities in specific region-to-region and after 2018 I used the avaerage of growth rate from 2012 to 2018 and added it for each year as a year growth rate to 2025
df_with_growth_rate=df_with_growth_rate.withColumn("tons_GrowthRate_12_14", col("tons_GrowthRate_12_13") + col("tons_GrowthRate_13_14"))
df_with_growth_rate=df_with_growth_rate.withColumn("tons_GrowthRate_12_15", col("tons_GrowthRate_12_14") + col("tons_GrowthRate_14_15"))
df_with_growth_rate=df_with_growth_rate.withColumn("tons_GrowthRate_12_16", col("tons_GrowthRate_12_15") + col("tons_GrowthRate_15_16"))
df_with_growth_rate=df_with_growth_rate.withColumn("tons_GrowthRate_12_17", col("tons_GrowthRate_12_16") + col("tons_GrowthRate_16_17"))
df_with_growth_rate=df_with_growth_rate.withColumn("tons_GrowthRate_12_18", col("tons_GrowthRate_12_17") + col("tons_GrowthRate_17_18"))
df_with_growth_rate=df_with_growth_rate.withColumn("tons_GrowthRate_12_19", col("tons_GrowthRate_12_18") + col("Avg_tons_GrowthRate_12_18"))
df_with_growth_rate=df_with_growth_rate.withColumn("tons_GrowthRate_12_20", col("tons_GrowthRate_12_19") + col("Avg_tons_GrowthRate_12_18"))
df_with_growth_rate=df_with_growth_rate.withColumn("tons_GrowthRate_12_21", col("tons_GrowthRate_12_20") + col("Avg_tons_GrowthRate_12_18"))
df_with_growth_rate=df_with_growth_rate.withColumn("tons_GrowthRate_12_22", col("tons_GrowthRate_12_21") + col("Avg_tons_GrowthRate_12_18"))
df_with_growth_rate=df_with_growth_rate.withColumn("tons_GrowthRate_12_23", col("tons_GrowthRate_12_22") + col("Avg_tons_GrowthRate_12_18"))
df_with_growth_rate=df_with_growth_rate.withColumn("tons_GrowthRate_12_24", col("tons_GrowthRate_12_23") + col("Avg_tons_GrowthRate_12_18"))
#df_with_growth_rate=df_with_growth_rate.withColumn("tons_GrowthRate_12_25", col("tons_GrowthRate_12_24") + col("Avg_tons_GrowthRate_12_18"))

In [0]:
#Extract the tables with columns I need to use for training and create regression model with creating year column
from pyspark.sql.functions import lit
df_ton_2013=df_with_growth_rate.select("dms_orig","dms_dest","dms_mode","sctg2","wgt_dist",col("tons_GrowthRate_12_13").alias("tons_GrowthRate"),col("tons_2013").alias("tons")).withColumn("year", lit(2013))
df_ton_2014=df_with_growth_rate.select("dms_orig","dms_dest","dms_mode","sctg2","wgt_dist",col("tons_GrowthRate_12_14").alias("tons_GrowthRate"),col("tons_2014").alias("tons")).withColumn("year", lit(2014))
df_ton_2015=df_with_growth_rate.select("dms_orig","dms_dest","dms_mode","sctg2","wgt_dist",col("tons_GrowthRate_12_15").alias("tons_GrowthRate"),col("tons_2015").alias("tons")).withColumn("year", lit(2015))
df_ton_2016=df_with_growth_rate.select("dms_orig","dms_dest","dms_mode","sctg2","wgt_dist",col("tons_GrowthRate_12_16").alias("tons_GrowthRate"),col("tons_2016").alias("tons")).withColumn("year", lit(2016))
df_ton_2017=df_with_growth_rate.select("dms_orig","dms_dest","dms_mode","sctg2","wgt_dist",col("tons_GrowthRate_12_17").alias("tons_GrowthRate"),col("tons_2017").alias("tons")).withColumn("year", lit(2017))
df_ton_2018=df_with_growth_rate.select("dms_orig","dms_dest","dms_mode","sctg2","wgt_dist",col("tons_GrowthRate_12_18").alias("tons_GrowthRate"),col("tons_2018").alias("tons")).withColumn("year", lit(2018))

In [0]:
#Union the datasets in one data
df_tons_train = df_ton_2013.union(df_ton_2014)
df_tons_train = df_tons_train.union(df_ton_2015)
df_tons_train = df_tons_train.union(df_ton_2016)
df_tons_train = df_tons_train.union(df_ton_2017)
df_tons_test = df_ton_2018

In [0]:
dms_orig_Numbers_df=df_tons_train.groupBy("dms_orig").agg(count("*").alias("dms_orig_Numbers")).select('dms_orig')
mvv_array = [int(row['dms_orig']) for row in dms_orig_Numbers_df.collect()]

In [0]:
mvv_array

Out[13]: [471,
 392,
 540,
 251,
 451,
 65,
 481,
 472,
 513,
 321,
 362,
 211,
 101,
 81,
 183,
 300,
 159,
 271,
 329,
 460,
 12,
 350,
 223,
 91,
 409,
 222,
 209,
 372,
 122,
 230,
 319,
 190,
 539,
 111,
 519,
 132,
 291,
 473,
 259,
 512,
 363,
 212,
 182,
 280,
 20,
 429,
 500,
 139,
 479,
 402,
 292,
 339,
 559,
 491,
 452,
 279,
 19,
 92,
 419,
 560,
 299,
 64,
 41,
 484,
 262,
 511,
 269,
 179,
 189,
 489,
 61,
 486,
 531,
 202,
 423,
 485,
 441,
 229,
 241,
 49,
 380,
 171,
 331,
 391,
 532,
 69,
 129,
 63,
 459,
 361,
 50,
 261,
 181,
 249,
 364,
 394,
 160,
 62,
 121,
 219,
 411,
 482,
 371,
 221,
 151,
 341,
 488,
 109,
 487,
 11,
 421,
 369,
 123,
 342,
 373,
 379,
 242,
 499,
 42,
 201,
 131,
 124,
 99,
 311,
 172,
 399,
 551,
 483,
 393,
 89,
 401,
 422]

In [0]:
chunks = [mvv_array[x:x+6] for x in range(0, len(mvv_array), 6)]

In [0]:
chunks

Out[16]: [[471, 392, 540, 251, 451, 65],
 [481, 472, 513, 321, 362, 211],
 [101, 81, 183, 300, 159, 271],
 [329, 460, 12, 350, 223, 91],
 [409, 222, 209, 372, 122, 230],
 [319, 190, 539, 111, 519, 132],
 [291, 473, 259, 512, 363, 212],
 [182, 280, 20, 429, 500, 139],
 [479, 402, 292, 339, 559, 491],
 [452, 279, 19, 92, 419, 560],
 [299, 64, 41, 484, 262, 511],
 [269, 179, 189, 489, 61, 486],
 [531, 202, 423, 485, 441, 229],
 [241, 49, 380, 171, 331, 391],
 [532, 69, 129, 63, 459, 361],
 [50, 261, 181, 249, 364, 394],
 [160, 62, 121, 219, 411, 482],
 [371, 221, 151, 341, 488, 109],
 [487, 11, 421, 369, 123, 342],
 [373, 379, 242, 499, 42, 201],
 [131, 124, 99, 311, 172, 399],
 [551, 483, 393, 89, 401, 422]]

In [0]:
chunks[0]

Out[17]: [471, 392, 540, 251, 451, 65]

In [0]:
df_tons_train_1=df_tons_train.filter(col('dms_orig').isin(chunks[0]))
df_tons_train_2=df_tons_train.filter(col('dms_orig').isin(chunks[1]))
df_tons_train_3=df_tons_train.filter(col('dms_orig').isin(chunks[2]))
df_tons_train_4=df_tons_train.filter(col('dms_orig').isin(chunks[3]))
df_tons_train_5=df_tons_train.filter(col('dms_orig').isin(chunks[4]))
df_tons_train_6=df_tons_train.filter(col('dms_orig').isin(chunks[5]))
df_tons_train_7=df_tons_train.filter(col('dms_orig').isin(chunks[6]))
df_tons_train_8=df_tons_train.filter(col('dms_orig').isin(chunks[7]))
df_tons_train_9=df_tons_train.filter(col('dms_orig').isin(chunks[8]))
df_tons_train_10=df_tons_train.filter(col('dms_orig').isin(chunks[9]))
df_tons_train_11=df_tons_train.filter(col('dms_orig').isin(chunks[10]))
df_tons_train_12=df_tons_train.filter(col('dms_orig').isin(chunks[11]))
df_tons_train_13=df_tons_train.filter(col('dms_orig').isin(chunks[12]))
df_tons_train_14=df_tons_train.filter(col('dms_orig').isin(chunks[13]))
df_tons_train_15=df_tons_train.filter(col('dms_orig').isin(chunks[14]))
df_tons_train_16=df_tons_train.filter(col('dms_orig').isin(chunks[15]))
df_tons_train_17=df_tons_train.filter(col('dms_orig').isin(chunks[16]))
df_tons_train_18=df_tons_train.filter(col('dms_orig').isin(chunks[17]))
df_tons_train_19=df_tons_train.filter(col('dms_orig').isin(chunks[18]))
df_tons_train_20=df_tons_train.filter(col('dms_orig').isin(chunks[19]))
df_tons_train_21=df_tons_train.filter(col('dms_orig').isin(chunks[20]))
df_tons_train_22=df_tons_train.filter(col('dms_orig').isin(chunks[21]))

In [0]:
df_tons_test_1=df_tons_test.filter(col('dms_orig').isin(chunks[0]))
df_tons_test_2=df_tons_test.filter(col('dms_orig').isin(chunks[1]))
df_tons_test_3=df_tons_test.filter(col('dms_orig').isin(chunks[2]))
df_tons_test_4=df_tons_test.filter(col('dms_orig').isin(chunks[3]))
df_tons_test_5=df_tons_test.filter(col('dms_orig').isin(chunks[4]))
df_tons_test_6=df_tons_test.filter(col('dms_orig').isin(chunks[5]))
df_tons_test_7=df_tons_test.filter(col('dms_orig').isin(chunks[6]))
df_tons_test_8=df_tons_test.filter(col('dms_orig').isin(chunks[7]))
df_tons_test_9=df_tons_test.filter(col('dms_orig').isin(chunks[8]))
df_tons_test_10=df_tons_test.filter(col('dms_orig').isin(chunks[9]))
df_tons_test_11=df_tons_test.filter(col('dms_orig').isin(chunks[10]))
df_tons_test_12=df_tons_test.filter(col('dms_orig').isin(chunks[11]))
df_tons_test_13=df_tons_test.filter(col('dms_orig').isin(chunks[12]))
df_tons_test_14=df_tons_test.filter(col('dms_orig').isin(chunks[13]))
df_tons_test_15=df_tons_test.filter(col('dms_orig').isin(chunks[14]))
df_tons_test_16=df_tons_test.filter(col('dms_orig').isin(chunks[15]))
df_tons_test_17=df_tons_test.filter(col('dms_orig').isin(chunks[16]))
df_tons_test_18=df_tons_test.filter(col('dms_orig').isin(chunks[17]))
df_tons_test_19=df_tons_test.filter(col('dms_orig').isin(chunks[18]))
df_tons_test_20=df_tons_test.filter(col('dms_orig').isin(chunks[19]))
df_tons_test_21=df_tons_test.filter(col('dms_orig').isin(chunks[20]))
df_tons_test_22=df_tons_test.filter(col('dms_orig').isin(chunks[21]))

In [0]:
dbutils.fs.rm("/FileStore/df",True)

Out[24]: True

In [0]:
df_tons_train_1.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_train_1.csv")
df_tons_train_2.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_train_2.csv")
df_tons_train_3.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_train_3.csv")
df_tons_train_4.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_train_4.csv")
df_tons_train_5.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_train_5.csv")
df_tons_train_6.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_train_6.csv")
df_tons_train_7.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_train_7.csv")
df_tons_train_8.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_train_8.csv")
df_tons_train_9.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_train_9.csv")
df_tons_train_10.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_train_10.csv")
df_tons_train_11.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_train_11.csv")
df_tons_train_12.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_train_12.csv")
df_tons_train_13.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_train_13.csv")
df_tons_train_14.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_train_14.csv")
df_tons_train_15.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_train_15.csv")
df_tons_train_16.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_train_16.csv")
df_tons_train_17.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_train_17.csv")
df_tons_train_18.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_train_18.csv")
df_tons_train_19.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_train_19.csv")
df_tons_train_20.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_train_20.csv")
df_tons_train_21.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_train_21.csv")
df_tons_train_22.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_train_22.csv")

In [0]:
df_tons_test_1.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_test_1.csv")
df_tons_test_2.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_test_2.csv")
df_tons_test_3.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_test_3.csv")
df_tons_test_4.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_test_4.csv")
df_tons_test_5.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_test_5.csv")
df_tons_test_6.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_test_6.csv")
df_tons_test_7.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_test_7.csv")
df_tons_test_8.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_test_8.csv")
df_tons_test_9.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_test_9.csv")
df_tons_test_10.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_test_10.csv")
df_tons_test_11.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_test_11.csv")
df_tons_test_12.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_test_12.csv")
df_tons_test_13.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_test_13.csv")
df_tons_test_14.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_test_14.csv")
df_tons_test_15.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_test_15.csv")
df_tons_test_16.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_test_16.csv")
df_tons_test_17.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_test_17.csv")
df_tons_test_18.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_test_18.csv")
df_tons_test_19.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_test_19.csv")
df_tons_test_20.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_test_20.csv")
df_tons_test_21.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_test_21.csv")
df_tons_test_22.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_test_22.csv")

In [0]:
#Extract the tables with columns I need to predict the tons for it
#df_ton_2019=df_with_growth_rate.select("dms_orig","dms_dest","dms_mode","sctg2","wgt_dist",col("tons_GrowthRate_12_19").alias("tons_GrowthRate")).withColumn("year", lit(2019))
#df_ton_2020=df_with_growth_rate.select("dms_orig","dms_dest","dms_mode","sctg2","wgt_dist",col("tons_GrowthRate_12_20").alias("tons_GrowthRate")).withColumn("year", lit(2020))
#df_ton_2021=df_with_growth_rate.select("dms_orig","dms_dest","dms_mode","sctg2","wgt_dist",col("tons_GrowthRate_12_21").alias("tons_GrowthRate")).withColumn("year", lit(2021))
#df_ton_2022=df_with_growth_rate.select("dms_orig","dms_dest","dms_mode","sctg2","wgt_dist",col("tons_GrowthRate_12_22").alias("tons_GrowthRate")).withColumn("year", lit(2022))
#df_ton_2023=df_with_growth_rate.select("dms_orig","dms_dest","dms_mode","sctg2","wgt_dist",col("tons_GrowthRate_12_23").alias("tons_GrowthRate")).withColumn("year", lit(2023))
df_ton_2024=df_with_growth_rate.select("dms_orig","dms_dest","dms_mode","sctg2","wgt_dist",col("tons_GrowthRate_12_24").alias("tons_GrowthRate")).withColumn("year", lit(2024))
#df_ton_2025=df_with_growth_rate.select("dms_orig","dms_dest","dms_mode","sctg2","wgt_dist",col("tons_GrowthRate_12_25").alias("tons_GrowthRate")).withColumn("year", lit(2025))


In [0]:
df_ton_2024.columns

Out[28]: ['dms_orig',
 'dms_dest',
 'dms_mode',
 'sctg2',
 'wgt_dist',
 'tons_GrowthRate',
 'year']

In [0]:
#Union the datasets in one data
#df_tons_predicted = df_ton_2019.union(df_ton_2020)
#df_tons_predicted = df_tons_predicted.union(df_ton_2021)
#df_tons_predicted = df_tons_predicted.union(df_ton_2022)
#df_tons_predicted = df_tons_predicted.union(df_ton_2023)
#df_tons_predicted = df_tons_predicted.union(df_ton_2024)
#df_tons_predicted = df_tons_predicted.union(df_ton_2025)

In [0]:
df_tons_predicted = df_ton_2024

In [0]:
df_tons_predicted_1=df_tons_predicted.filter(col('dms_orig').isin(chunks[0]))
df_tons_predicted_2=df_tons_predicted.filter(col('dms_orig').isin(chunks[1]))
df_tons_predicted_3=df_tons_predicted.filter(col('dms_orig').isin(chunks[2]))
df_tons_predicted_4=df_tons_predicted.filter(col('dms_orig').isin(chunks[3]))
df_tons_predicted_5=df_tons_predicted.filter(col('dms_orig').isin(chunks[4]))
df_tons_predicted_6=df_tons_predicted.filter(col('dms_orig').isin(chunks[5]))
df_tons_predicted_7=df_tons_predicted.filter(col('dms_orig').isin(chunks[6]))
df_tons_predicted_8=df_tons_predicted.filter(col('dms_orig').isin(chunks[7]))
df_tons_predicted_9=df_tons_predicted.filter(col('dms_orig').isin(chunks[8]))
df_tons_predicted_10=df_tons_predicted.filter(col('dms_orig').isin(chunks[9]))
df_tons_predicted_11=df_tons_predicted.filter(col('dms_orig').isin(chunks[10]))
df_tons_predicted_12=df_tons_predicted.filter(col('dms_orig').isin(chunks[11]))
df_tons_predicted_13=df_tons_predicted.filter(col('dms_orig').isin(chunks[12]))
df_tons_predicted_14=df_tons_predicted.filter(col('dms_orig').isin(chunks[13]))
df_tons_predicted_15=df_tons_predicted.filter(col('dms_orig').isin(chunks[14]))
df_tons_predicted_16=df_tons_predicted.filter(col('dms_orig').isin(chunks[15]))
df_tons_predicted_17=df_tons_predicted.filter(col('dms_orig').isin(chunks[16]))
df_tons_predicted_18=df_tons_predicted.filter(col('dms_orig').isin(chunks[17]))
df_tons_predicted_19=df_tons_predicted.filter(col('dms_orig').isin(chunks[18]))
df_tons_predicted_20=df_tons_predicted.filter(col('dms_orig').isin(chunks[19]))
df_tons_predicted_21=df_tons_predicted.filter(col('dms_orig').isin(chunks[20]))
df_tons_predicted_22=df_tons_predicted.filter(col('dms_orig').isin(chunks[21]))

In [0]:
dbutils.fs.rm("/FileStore/df",True)

Out[32]: True

In [0]:
df_tons_predicted_1.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_predicted_1.csv")
df_tons_predicted_2.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_predicted_2.csv")
df_tons_predicted_3.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_predicted_3.csv")
df_tons_predicted_4.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_predicted_4.csv")
df_tons_predicted_5.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_predicted_5.csv")
df_tons_predicted_6.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_predicted_6.csv")
df_tons_predicted_7.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_predicted_7.csv")
df_tons_predicted_8.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_predicted_8.csv")
df_tons_predicted_9.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_predicted_9.csv")
df_tons_predicted_10.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_predicted_10.csv")
df_tons_predicted_11.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_predicted_11.csv")
df_tons_predicted_12.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_predicted_12.csv")
df_tons_predicted_13.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_predicted_13.csv")
df_tons_predicted_14.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_predicted_14.csv")
df_tons_predicted_15.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_predicted_15.csv")
df_tons_predicted_16.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_predicted_16.csv")
df_tons_predicted_17.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_predicted_17.csv")
df_tons_predicted_18.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_predicted_18.csv")
df_tons_predicted_19.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_predicted_19.csv")
df_tons_predicted_20.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_predicted_20.csv")
df_tons_predicted_21.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_predicted_21.csv")
df_tons_predicted_22.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/df_tons_predicted_22.csv")